In [ ]:
import tensorflow as tf
from keras.models import load_model
from PIL import Image
import numpy as np
import base64
import io
from flask import Flask, request, jsonify, render_template

app = Flask(__name__)
# Modeli yükle
model = load_model('MB4.h5')
# Katman isimlerini al
layer_names = [layer.name for layer in model.layers if isinstance(layer, tf.keras.layers.Conv2D)]
# Görselleştirme modelini oluştur
visualization_model = tf.keras.models.Model(inputs=model.input, outputs=[model.get_layer(name).output for name in layer_names])

# API endpoint'i
@app.route('/', methods=['GET', 'POST'])
def katman_ciktilari():
    # Gelen görüntüyü al
    if request.method == 'POST':
        img = Image.open(io.BytesIO(file.read()))
        img = img.convert('L')
        img_array_resized = np.array(img.resize((model.input_shape[1], model.input_shape[2])))
        img_tensor = np.expand_dims(img_array_resized, axis=0)
        img_tensor = img_tensor.astype(np.float32)

        # Katman çıktılarını al
        activations = visualization_model.predict(img_tensor)

        classes = ["airplane", "automobile", "bird", "cat", "dog", "frog", "horse", "ship", "truck"]
        predictions = model.predict(img_tensor)
        predicted_class = classes[np.argmax(predictions)]

        print("Predicted class:", predicted_class)

        # Katman çıktılarını resim olarak dönüştür
        results = []
        for layer_name, feature_map in zip(layer_names, activations):
            if len(feature_map.shape) == 4:
                n_features = feature_map.shape[-1]
                size = feature_map.shape[1]
                display_grid = np.zeros((size, size * n_features), dtype=np.uint8)
                for i in range(n_features):
                    x = feature_map[0, :, :, i]
                    x -= x.mean()
                    x /= x.std()
                    x *= 64
                    x += 128
                    x = np.clip(x, 0, 255).astype('uint8')
                    display_grid[:, i * size: (i + 1) * size] = x

                # Oluşturulan grid'i base64 formatında dönüştür
                with io.BytesIO() as byte_io:
                    resim = Image.fromarray(display_grid)
                    resim.save(byte_io, 'JPEG')
                    byte_data = byte_io.getvalue()
                    base64_data = base64.b64encode(byte_data).decode('ascii')
                    results.append({'name': layer_name, 'image': base64_data})

        return render_template('results.html', results=results)

    return "Welcome to the API!"


if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.102:5000
Press CTRL+C to quit


In [ ]:
%tb